In [1]:
import tensorflow as tf
import numpy as np

Data importing

In [213]:
audio = np.loadtxt('Audiobooks_data.csv', delimiter = ',')

In [215]:
audio

array([[9.9400e+02, 1.6200e+03, 1.6200e+03, ..., 5.0000e+00, 9.2000e+01,
        0.0000e+00],
       [1.1430e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0590e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 3.8800e+02,
        0.0000e+00],
       ...,
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00],
       [2.5100e+02, 1.6740e+03, 3.3480e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [217]:
audio_inputs = audio[:, :-1]
audio_targets = audio[:, -1]

In [219]:
audio_targets.shape[0]

14084

In [221]:
np.sum(audio_targets)

2237.0

Balancing the targets

In [224]:
target_zeros = np.sum(audio_targets)
extra_zeros = []
zero_counter = 0

for i in range(audio_targets.shape[0]):
    if audio_targets[i] == 0:
        zero_counter = zero_counter + 1
        if zero_counter > target_zeros :
            extra_zeros.append(i)

In [226]:
audio_inputs = np.delete(audio_inputs,extra_zeros, axis= 0)
audio_targets = np.delete(audio_targets,extra_zeros, axis = 0)

In [228]:
audio_inputs.shape[0] 

4474

In [230]:
audio_targets.sum()

2237.0

Scaling the data

In [234]:
from sklearn import preprocessing

Shuffle the data 

In [237]:
suffle_indices =np.arange(audio_targets.shape[0])

np.random.shuffle(suffle_indices)

audio_inputs = audio_inputs[suffle_indices]
audio_targets = audio_targets[suffle_indices]

In [239]:
audio_inputs = preprocessing.scale(audio_inputs)

Splitting data

In [242]:
sample_data = audio_inputs.shape[0]

sample_train = int(sample_data * 0.8)
sample_valid = int(sample_data * 0.1)
sample_test = int(sample_data * 0.1)

train_data_inputs = audio_inputs[:sample_train,]
valid_data_inputs = audio_inputs[sample_train : sample_train+ sample_valid,]
test_data_inputs = audio_inputs[sample_train + sample_valid : ,]

train_data_targets = audio_targets[:sample_train,]
valid_data_targets = audio_targets[sample_train : sample_train+ sample_valid,]
test_data_targets = audio_targets[sample_train + sample_valid : ,]


In [244]:
print(f'Train data {train_data_inputs.shape} \n Validation data {valid_data_inputs.shape} \n Test data {test_data_inputs.shape}')

Train data (3579, 11) 
 Validation data (447, 11) 
 Test data (448, 11)


Saving the files  as  Tensors

In [247]:
np.savez('audio_train', inputs = train_data_inputs, targets = train_data_targets)
np.savez('audio_valid', inputs = valid_data_inputs, targets = valid_data_targets)
np.savez('audio_test', inputs = test_data_inputs, targets = test_data_targets)

Now let's again load the data which we saved it already as Tensors

In [250]:
npz = np.load('audio_train.npz')
train_inputs = npz['inputs']
train_targets = npz['targets']

In [297]:
npz = np.load('audio_valid.npz')
valid_inputs, valid_targets = npz['inputs'], npz['targets']

npz = np.load('audio_test.npz')
test_inputs,test_targets = npz['inputs'] , npz['targets']

Define Hyper parameters

In [328]:
output_size = 2
hidden_layers = 50

Build a model

In [330]:
model = tf.keras.Sequential([

                            tf.keras.layers.Dense(hidden_layers, activation = 'relu'),
                            tf.keras.layers.Dropout(0.3),
                            tf.keras.layers.Dense(hidden_layers, activation = 'relu'),
                            tf.keras.layers.Dropout(0.3),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
    ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics = 'accuracy')

In [ ]:
batch_size = 100
epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 5,restore_best_weights=True)

model.fit(
     train_inputs,
    train_targets,
    batch_size= batch_size,
    epochs = epochs,
    callbacks = early_stopping,
    validation_data = (valid_inputs, valid_targets),
    verbose = 2
    
)

Epoch 1/100
36/36 - 2s - loss: 0.6367 - accuracy: 0.6102 - val_loss: 0.5366 - val_accuracy: 0.7271 - 2s/epoch - 56ms/step
Epoch 2/100
36/36 - 0s - loss: 0.5342 - accuracy: 0.7047 - val_loss: 0.4803 - val_accuracy: 0.7450 - 287ms/epoch - 8ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4935 - accuracy: 0.7105 - val_loss: 0.4440 - val_accuracy: 0.7539 - 300ms/epoch - 8ms/step
Epoch 4/100
36/36 - 0s - loss: 0.4652 - accuracy: 0.7404 - val_loss: 0.4233 - val_accuracy: 0.7629 - 307ms/epoch - 9ms/step
Epoch 5/100
36/36 - 0s - loss: 0.4523 - accuracy: 0.7407 - val_loss: 0.4132 - val_accuracy: 0.7606 - 287ms/epoch - 8ms/step
Epoch 6/100
36/36 - 0s - loss: 0.4344 - accuracy: 0.7555 - val_loss: 0.4003 - val_accuracy: 0.7808 - 283ms/epoch - 8ms/step
Epoch 7/100
36/36 - 0s - loss: 0.4303 - accuracy: 0.7524 - val_loss: 0.3972 - val_accuracy: 0.7696 - 263ms/epoch - 7ms/step
Epoch 8/100
36/36 - 0s - loss: 0.4265 - accuracy: 0.7533 - val_loss: 0.3916 - val_accuracy: 0.7606 - 240ms/epoch - 7ms/step
Epoch 9/10

In [311]:
loss, accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 5ms/step - loss: 0.3215 - accuracy: 0.8125


In [319]:
print(f' Model Details:\n Accuracy: {accuracy * 100}')

 Model Details:
 Accuracy: 81.25
